# **Trabalho 3**

Todos os anos centenas de filmes são lançados no mundo todo por diversas produtoras que contam com orçamentos de todos os tamanhos, gerando uma indústria bilhonária. Nesse contexto, você como cientista da computação recém formado foi contratado para monitorar em tempo real as impressões da platéia por meio de posts em redes sociais.

Basicamente o seu trabalho é monitorar as impressões positivas e negativas de um determinado filme, nos últimos dias (exemplo: 30 dias). Você poderá fazer isso lendo todas as postagens e interpretando cada uma como positiva ou negativa ou **usando** o conhecimento adquirido durante a graduação e, portanto, automatizar essa tarefa. **Faça a escolha correta** e automatize essa tarefa. 

Para auxiliar nessa tarefa, considere o dataset anterior e os conceitos estudados em sala de aula.

Escolha três filmes* já lançados de sua preferência para conduzir essa análise. Lembre-se que seu trabalho é apresentar uma forma automática de entender impressões positivas e negativas sobre os filmes escolhidos a partir de posts em uma rede social (no caso, o Twitter).

O que você precisa apresentar é:

+ uma análise sobre os posts referentes aos filmes escolhidos (isto é, o notebbok contendo o código e comentado. Seja organizado). Por exemplo, espera-se que no mínimo você apresente um gráfico de impressões positivas e negativas sobre cada filme escolhido nos últimos **n** dias.

+ fazer um vídeo de no máximo 5 minutos explicando as tuas principais estratégias.

**Data de entrega:** 10 de agosto de 2022

*Seja prudente na escolha dos filmes, de modo que haja comentários suficientes para a sua análise.

## Dataset

In [3]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 53.4 MB/s 
     |████████████████████████████████| 981 kB 62.3 MB/s 
     |████████████████████████████████| 263 kB 69.6 MB/s 
     |████████████████████████████████| 3.8 MB 36.8 MB/s 
     |████████████████████████████████| 1.3 MB 55.4 MB/s 
     |████████████████████████████████| 468 kB 44.3 MB/s 
     |████████████████████████████████| 880 kB 52.2 MB/s 
     |████████████████████████████████| 6.6 MB 5.8 MB/s 
     |████████████████████████████████| 596 kB 27.5 MB/s 
     |████████████████████████████████| 101 kB 7.9 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.7-py3-none-any.whl size=25312842 sha256=4282da5908a7e7fca0d8d28454e25f1c01366b881e4015dafcbe4a8aa4bbfc4e
  Stored in directory: /root/.cache/pip/wheels/e5/98/8e/ce355dcb92451e85fab93f7ea2da068843e93e703928cd06fb
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py

In [4]:
import pandas as pd
import io
import requests
import keras
import ktrain
import json
import textblob
import wordcloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re    
import tweepy
from ktrain import text

In [5]:
s = requests.get('https://raw.githubusercontent.com/ragero/text-collections/master/complete_texts_csvs/review_polarity.csv').content

dataset = pd.read_csv(io.StringIO(s.decode('utf-8')),
                na_values=['?'],
                skipinitialspace = True)
dataset = dataset.sample(frac=1)

In [6]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(dataset, 
                                                                   'text',
                                                                   label_columns='class',
                                                                   maxlen=64, 
                                                                   max_features=50000,
                                                                   preprocess_mode='bert',
                                                                   lang=None,
                                                                   val_pct = 0.1,
                                                                   )

['neg', 'pos']
      neg  pos
1278  0.0  1.0
685   1.0  0.0
1892  0.0  1.0
1243  0.0  1.0
541   1.0  0.0
['neg', 'pos']
      neg  pos
1431  0.0  1.0
1556  0.0  1.0
601   1.0  0.0
300   1.0  0.0
569   1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
model = text.text_classifier('bert', (X_train, y_train) , preproc=preprocess)
classifier = ktrain.get_learner(model, 
                             train_data=(X_train, y_train), 
                             val_data=(X_test, y_test),
                             batch_size=64
                             )

Is Multi-Label? False
maxlen is 64
done.


In [20]:
classifier.fit_onecycle(0.00002,8)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/8
29/29 [==============================] - 24s 821ms/step - loss: 0.1841 - accuracy: 0.9433 - val_loss: 0.8306 - val_accuracy: 0.6500
Epoch 2/8
29/29 [==============================] - 24s 836ms/step - loss: 0.1222 - accuracy: 0.9678 - val_loss: 0.9131 - val_accuracy: 0.6400
Epoch 3/8
29/29 [==============================] - 24s 827ms/step - loss: 0.0772 - accuracy: 0.9806 - val_loss: 1.1322 - val_accuracy: 0.6250
Epoch 4/8
29/29 [==============================] - 24s 818ms/step - loss: 0.1155 - accuracy: 0.9594 - val_loss: 1.2301 - val_accuracy: 0.6550
Epoch 5/8
29/29 [==============================] - 24s 822ms/step - loss: 0.0616 - accuracy: 0.9806 - val_loss: 1.3984 - val_accuracy: 0.6350
Epoch 6/8
29/29 [==============================] - 24s 824ms/step - loss: 0.0467 - accuracy: 0.9867 - val_loss: 1.2605 - val_accuracy: 0.6500
Epoch 7/8
29/29 [==============================] - 24s 822ms/step - loss: 0.0496 - ac

In [21]:
classifier.validate()

              precision    recall  f1-score   support

           0       0.68      0.63      0.65       102
           1       0.64      0.69      0.67        98

    accuracy                           0.66       200
   macro avg       0.66      0.66      0.66       200
weighted avg       0.66      0.66      0.66       200



array([[64, 38],
       [30, 68]])

## Aplicando a novos reviews

In [22]:
predictor = ktrain.get_predictor(classifier.model, preprocess)

In [26]:
predictor.predict('this is a good movie')

'pos'

In [2]:
log=pd.read_csv('keys.csv')
consumerkey=log['key'][0]
consumersecret=log['key'][1]
accesstoken=log['key'][2]
accesstokensecret=log['key'][3]

In [45]:
client = tweepy.Client(accesstoken)

#Lendo twetts dos filmes thor amor e trovao, Sonic e Elvis.
res1 =  tweepy.Paginator(client.search_recent_tweets, query="thorloveandthunder", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
res2 =  tweepy.Paginator(client.search_recent_tweets, query="Sonic", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)
res3 =  tweepy.Paginator(client.search_recent_tweets, query="elvis", tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=200)

In [46]:
def remove_caracteres(text,carc):
    for i in range(len(carc)):
            text = text.replace(carc[i],"")
    return text

def remove_stop_words(text,stops):
    for i in stops:
            idx = text.find(i)
            if(idx != -1):
                text = text[0:idx]+text[(idx+len(i)):len(text)]
    return text

In [47]:
#Guardando os twetts em im arquivo json
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
stopwords.update(["RT","thor","thunder","ThorLoveAndThunder","https","\n"])
caracteres = "#@:;|_{}[]()"
info = {"thor":{"text":[]},"Sonic":{"text":[]},"elvis":{"text":[]}}
with open('movies.json', 'w') as outfile:
    for response in res1:
        response.text = remove_caracteres(response.text,caracteres)
        response.text = remove_stop_words(response.text,stopwords)
        info["thor"]["text"].append(response.text)
    for response2 in res2:
        response2.text = remove_caracteres(response2.text,caracteres)
        response2.text = remove_stop_words(response2.text,stopwords)
        info["Sonic"]["text"].append(response2.text)
    for response3 in res3:
        response3.text = remove_caracteres(response3.text,caracteres)
        response3.text = remove_stop_words(response3.text,stopwords)
        info["elvis"]["text"].append(response3.text)
    outfile.write(json.dumps(info))

In [ ]:
# Word Cloud com os textos dos tweets
wordcloud = WordCloud(width=1600, stopwords=stopwords,height=800,max_font_size=200,max_words=20,collocations=False, background_color='white').generate(string)
plt.figure(figsize=(40,30))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## **Notebook útil e bem interessante que pode ajudar no trabalho**
https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_natural-language-processing-applications/sentiment-analysis-and-dataset.ipynb